# Run Delphes and extract observables

Johann Brehmer, Kyle Cranmer, Marco Farina, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import os
import math
from collections import OrderedDict

from madminer.delphes import DelphesProcessor
from madminer.sampling import combine_and_shuffle
from madminer.utils.particle import MadMinerParticle

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')


In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma_sys/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma_sys/'
log_dir = base_dir + 'logs/wgamma_sys/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'

## Observables and cuts

In [4]:
def calculate_mt(leptons, photons, jets, met):
    # Particles
    if len(leptons) < 1:
        raise RuntimeError()
    
    l = leptons[0]
    
    # Transverse mass and Delta
    cos_delta_phi = np.cos(l.phi() - met.phi())
    mt = (2 * l.pt * met.pt * (1. - cos_delta_phi))**0.5
    
    return mt

In [5]:
def calculate_phi(leptons, photons, jets, met, eta_solution=0):
    # Parameters
    mw = 80.4
    
    # Particles
    if len(leptons) < 1 or len(photons) < 1:
        raise RuntimeError()
    
    l = leptons[0]
    a = photons[0]
    
    # Transverse mass and Delta
    mt = calculate_mt(leptons, photons, jets, met)
    deltasq = 0.
    if met.pt > 0. and l.pt > 0.:
        deltasq = (mw**2 - mt**2) / (2. * met.pt * l.pt)
    
    # v reconstruction, "normal" case
    if deltasq > 0.:
        # Two solutions
        temp = np.log(1 + deltasq**0.5 * (2 + deltasq)**0.5 + deltasq)
        eta_v_plus = l.eta + temp
        eta_v_minus = l.eta - temp
        
        # Select one of them
        if eta_solution > 0:
            eta_v = eta_v_plus
        elif eta_solution < 0:
            eta_v = eta_v_minus
        else:
            dice = np.random.rand()
            if dice > 0.5:
                eta_v = eta_v_plus
            else:
                eta_v = eta_v_minus
            
    # v reconstruction, "other" case
    else:
        eta_v = l.eta
        
    # v particle
    v = MadMinerParticle()
    v.setptetaphim(met.pt, eta_v, met.phi(), 0.)
    
    # W and Wgamma reconstruction
    w = l + v
    vv = w + a
    
    # Boost into VV frame
    v_ = v.boost(vv.boostvector)
    l_ = l.boost(vv.boostvector)
    a_ = a.boost(vv.boostvector)
    w_ = w.boost(vv.boostvector)
    r_ = vv # vv.boost(vv.boostvector)

    # Calculate axes of "special frame" (1708.07823)
    z_ = w_.vector.unit()
    x_ = (r_.vector - z_ * r_.vector.dot(z_)).unit()
    y_ = z_.cross(x_)
    
    # Calculate x and y components of lepton wrt special x_, y_, z_ system
    lx_ = l_.vector.dot(x_)
    ly_ = l_.vector.dot(y_)
    
    # Calculate phi
    phi = math.atan2(ly_, lx_)
    
    return phi
    
    
def calculate_phi_minus(leptons, photons, jets, met):
    return calculate_phi(leptons, photons, jets, met, eta_solution=-1)
    
    
def calculate_phi_plus(leptons, photons, jets, met):
    return calculate_phi(leptons, photons, jets, met, eta_solution=1)

In [6]:
def setup_observables(delphesprocessor):
    delphesprocessor.reset_observables()
        
    # Default observables (four-momenta of a, l, j, and met)
    delphesprocessor.add_default_observables(
        n_leptons_max=1,
        n_photons_max=1,
        n_jets_max=1,
        include_charge=False,
        include_numbers=False,
        include_met=True
    )
    
    # Lepton flavour
    delphesprocessor.add_observable('pdgid_l1', 'l[0].pdgid', required=True)

    # Two-particle systems
    delphesprocessor.add_observable('m_la', '(l[0] + a[0]).m', required=True)
    delphesprocessor.add_observable('m_lmet', '(l[0] + met).m', required=True)
    delphesprocessor.add_observable('m_amet', '(a[0] + met).m', required=True)
    delphesprocessor.add_observable('pt_la','(l[0] + a[0]).pt', required=True)
    delphesprocessor.add_observable('pt_lmet','(l[0] + met).pt', required=True)
    delphesprocessor.add_observable('pt_amet','(a[0] + met).pt', required=True)
    delphesprocessor.add_observable('deltaphi_la', 'l[0].phi() - a[0].phi()', required=True)
    delphesprocessor.add_observable('deltaphi_lmet', 'l[0].phi() - met.phi()', required=True)
    delphesprocessor.add_observable('deltaphi_amet', 'a[0].phi() - met.phi()', required=True)
    delphesprocessor.add_observable('deltaeta_la', 'l[0].eta - a[0].eta', required=True)

    # Three-particle system
    delphesprocessor.add_observable('m_almet', '(a[0] + l[0] + met).m', required=True)
    delphesprocessor.add_observable('pt_almet', '(a[0] + l[0] + met).pt', required=True)
    
    # mT(W) and ressurrection phi
    delphesprocessor.add_observable_from_function('mt', calculate_mt, required=True)
    delphesprocessor.add_observable_from_function('phi_minus', calculate_phi_minus, required=True)
    delphesprocessor.add_observable_from_function('phi_plus', calculate_phi_plus, required=True)
    delphesprocessor.add_observable_from_function('phi', calculate_phi, required=True)


In [7]:
def setup_cuts(delphesprocessor, use_tight_cuts=False):
    delphesprocessor.reset_cuts()
    
    if use_tight_cuts:
        delphesprocessor.add_cut(
            'pt_a1 >= 300.'
        )
        delphesprocessor.add_cut(
            'pt_l1 >= 80.'
        )
        delphesprocessor.add_cut(
            'et_miss >= 80.'
        )
        delphesprocessor.add_cut(
            '(deltaphi_la**2 + deltaeta_la**2)**0.5 >= 3.'
        )
        delphesprocessor.add_cut(
            'eta_l1**2 < 2.4**2'
        )
        
    else:
        delphesprocessor.add_cut(
            'pt_a1 >= 20.'
        )
        delphesprocessor.add_cut(
            'pt_l1 >= 20.'
        )
        delphesprocessor.add_cut(
            'et_miss >= 20.'
        )

## Main loop

In [8]:
n_runs_per_benchmark = 1  # 30  # Number of run_cards

In [9]:
run = 0

for i_card in range(n_runs_per_benchmark):
    logging.info('Starting analysis of runs for card {}'.format(i_card))
            
    # Load setup
    dp = DelphesProcessor(sample_dir + 'setup.h5', debug=True)
    
    # Load events
    for benchmark in ['sm']:
        run += 1
        run_str = str(run)
        if len(run_str) < 2:
            run_str = '0' + run_str

        #dp.add_hepmc_sample(
        #    mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc.gz'.format(run_str),
        #    sampled_from_benchmark=benchmark
        #)
        dp.add_delphes_sample(
            mg_process_dir + 'Events/run_{}/tag_1_pythia8_events_delphes.root'.format(run_str),
            mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc.gz'.format(run_str),
            sampled_from_benchmark=benchmark
        )
        
    # Run Delphes
    dp.run_delphes(
        delphes_directory=delphes_dir,
        delphes_card=card_dir + 'delphes_card.dat',
        log_file=log_dir + '/delphes_{}.log'.format(run - 1),
        initial_command='source activate python2',
    )
    
    # Tight analysis
    setup_observables(dp)
    setup_cuts(dp, use_tight_cuts=True)
    
    dp.analyse_delphes_samples(delete_delphes_files=False)
    
    dp.save(sample_dir + 'samples_tight_{}.h5'.format(i_card))
    
    # Loose analysis
    setup_observables(dp)
    setup_cuts(dp)
    
    dp.analyse_delphes_samples(delete_delphes_files=False)
    
    dp.save(sample_dir + 'samples_{}.h5'.format(i_card))


09:46  
09:46  ------------------------------------------------------------
09:46  |                                                          |
09:46  |  MadMiner v0.1.1                                         |
09:46  |                                                          |
09:46  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
09:46  |                                                          |
09:46  ------------------------------------------------------------
09:46  
09:46  Benchmarks found in MadMiner file: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
09:46  Adding Delphes sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma_sys/Events/run_01/tag_1_pythia8_events_delphes.root, based on HepMC file at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma_sys/Events/run_01/tag_1_pythia8_events.hepmc.gz


KeyboardInterrupt: 

## Combine samples

In [ ]:
filenames_in = [sample_dir + 'samples_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples.h5')

In [ ]:
filenames_in = [sample_dir + 'samples_tight_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples_tight.h5')